In [12]:
import os
WORK = os.environ["WORK"]
%cd $WORK

/lustre/work/chaselab/malyetama


In [6]:
from glob import glob
from PIL import Image
import os
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm
from pathlib import Path
import imageio
from pygifsicle import optimize
from IPython.display import Markdown, display
import dotenv
import base64
import requests


def create_fakes_gif(
    DATASET_NAME, subset=None, output_dir=None, display_gif=False, verbose=False):
    
    def process(i):
        im = Image.open(i)
        if DATASET_NAME == 'metfaces':
            left, top, right, bottom = 0, 0, 1020 * 2, 1020 * 2
        else:
            left, top, right, bottom = 0, 0, 1020, 1020
        im_cropped = im.crop((left, top, right, bottom))
        return im_cropped.save(f'{history}/{Path(i).stem}.jpg')
    
    def upload_img(image, token):
        with open(image, "rb") as file:
            url = "https://api.imgbb.com/1/upload"
            parameters = {
                "key": token,
                "image": base64.b64encode(file.read()),
            }
            res = requests.post(url, parameters)
            link = res.json()
            url = link['data']['url']
            return url

    WORK = os.environ["WORK"]
    PROJ_DIR = f'{WORK}/ADA_Project'
    
    dotenv.load_dotenv(f'{PROJ_DIR}/.env')
    token = os.getenv('TOKEN')

    TRfolders = f'{PROJ_DIR}/training_runs/'
    TRfolders_ = glob(f'{PROJ_DIR}/training_runs/*')
    datasets = [
        x.replace(TRfolders, '').replace('_training-runs', '')
        for x in TRfolders_
    ]
    ds_rename = lambda before, after: [after if x == before else x for x in datasets]
    datasets = ds_rename('AFHQ', 'AFHQ-CAT')
    datasets = ds_rename('FFHQ', 'FFHQ_FULL')
    
    if verbose:
        print(f'Available datasets:\n {datasets}')

    d = {}

    for folder, dataset in zip(TRfolders_, datasets):
        files = sorted(glob(folder + "/**/*"))
        fakes = [x for x in files if 'fakes' in x]
        if fakes == []:
            continue
        d[dataset] = {}
        d[dataset]['files'] = fakes[::subset]
    
    history = f'{PROJ_DIR}/datasets/{DATASET_NAME}_history'
    try:
        shutil.rmtree(history)
    except:
        pass
    Path(history).mkdir(exist_ok=True)
    

    n_jobs = multiprocessing.cpu_count() - 1

    _ = Parallel(n_jobs=n_jobs)(delayed(process)(i)
                                     for i in tqdm(d[DATASET_NAME]['files']))

    history_imgs = sorted([x for x in glob(f'{history}/*.jpg')])
    history_imgs = [history_imgs[-1]] + [x for x in history_imgs if 'init' not in x]
    
    if subset is not None and verbose:
        print(f'Subset size: {len(history_imgs)} image')
    
    if output_dir is None:
        output_dir = Path.cwd()
        anim_file = f'{output_dir}/{DATASET_NAME}.gif'
    anim_file = f'{DATASET_NAME}.gif'

    with imageio.get_writer(anim_file, mode='I') as writer:
        for filename in history_imgs:
            image = imageio.imread(filename)
            if filename == history_imgs[-1]:
                for _ in range(15):
                    writer.append_data(image)
            writer.append_data(image)
        image = imageio.imread(filename)
        writer.append_data(image)
    
    file_size = lambda file: Path(file).stat().st_size / 1e+6
    if verbose:
        print(f'gif size before optimization: {file_size(anim_file):.2f} MB')
    optimize(source=anim_file, destination=anim_file)
    if verbose:
        print(f'         after optimization: {file_size(anim_file):.2f} MB')
    
    if display_gif is True:
        print('Loading...')
        img_url = upload_img(anim_file, token)
        display(Markdown(f'![]({img_url})'))


In [7]:
create_fakes_gif(DATASET_NAME='StyleGAN2_WILD-AFHQ', display_gif=True, verbose=True)

  0%|          | 0/45 [00:00<?, ?it/s]

Available datasets:
 ['AFHQ-WILD', 'AFHQ-CAT', 'metfaces', 'StyleGAN2_metfaces', 'POKEMON', 'StanfordDogs', 'StyleGAN2_AFHQ-DOG', 'cars196', 'AFHQ-DOG', 'FFHQ_5K', 'FFHQ_2K', 'ANIME-FACES', 'StyleGAN2_WILD-AFHQ', '102flowers', 'FFHQ_30K', 'FFHQ_FULL']


100%|██████████| 45/45 [00:04<00:00, 10.18it/s]


gif size before optimization: 60.10 MB
         after optimization: 28.20 MB
Loading...


![](https://i.ibb.co/MCm73tb/d30ae1edd07b.gif)